In [11]:
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup
from flask import Flask, render_template, redirect
import os


In [14]:

def init_browser():
    executable_path = {"executable_path": os.getcwd() + "/chromedriver"} #{"executable_path": "/Users/Naveen/Desktop/Data_BootCamp/class_work/2_9/chromedriver"}
    return Browser("chrome", **executable_path, headless=False)



In [3]:
def scrapenews():
    listings = {}
    
    brrowser = init_browser()
    url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
    brrowser.visit(url)

    html = brrowser.html
    soup = BeautifulSoup(html, "html.parser")
    
    
    chunks = soup.find_all("div", class_ = "list_text")
        
    for chunk in chunks:
        
        header = chunk.find('a').text
            
        body = chunk.find("div", class_ = "article_teaser_body").text
        
        listings[header] = body
    brrowser.quit()
    news = []
    news.append(list(listings.keys())[0])
    news.append(list(listings.values())[0])
    
    return news

In [4]:
def scrapeimg():

    brrowser = init_browser()
    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    brrowser.visit(url)

    html = brrowser.html
    soup = BeautifulSoup(html, "html.parser")
    
    
    chunk = soup.find("div", class_ = "carousel_container")
    link = chunk.find("a")["data-fancybox-href"]
    imagelink = "https://www.jpl.nasa.gov" + str(link)

    brrowser.quit()
        
    return imagelink


In [5]:
def scrapeweather():
    weatherurl = "https://twitter.com/marswxreport?lang=en"

    browser = init_browser()
    browser.visit(weatherurl)
    
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    
    chunk = soup.find("div", class_ = "js-tweet-text-container")
    text = chunk.find("p").text
    #browser.quit()
    return text

In [15]:
#test

weatherText = scrapeweather()
print(weatherText)

InSight sol 119 (2019-03-28) low -97.4ºC (-143.3ºF) high -15.0ºC (4.9ºF)
winds from the SW at 4.0 m/s (9.0 mph) gusting to 9.9 m/s (22.2 mph)
pressure at 7.30 hPapic.twitter.com/d5nSamjabl


In [98]:
def scrapehemi():
    
    url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    hemilist = {}
    browser = init_browser()
    browser.visit(url)
    
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    
    
    chunks = soup.find_all("div", class_ = "item")
    browser.quit()

    for chunk in chunks:
       
        title = chunk.find('h3').text
        
        imglink = chunk.find('a', class_ = "itemLink product-item")["href"]
        #imglink = chunk.find("img")["src"]
        hemiurl = "https://astrogeology.usgs.gov" + imglink       
        #the image on the given page is lame and small, so I did a tedious thing and rerouted to the main page for each hemisphere to
        #scrape the image on that page instead. 
        hemibrowser = init_browser()
        hemibrowser.visit(hemiurl)
        hemihtml = hemibrowser.html
        
        hemisoup = BeautifulSoup(hemihtml, "html.parser")
        #hemidiv = hemisoup.find("div", class_ ="container")
        hemilink = hemisoup.find('img', class_ = "wide-image")["src"]
        hemilist[title] = "https://astrogeology.usgs.gov" + hemilink
        hemibrowser.quit()
        
    return hemilist

In [90]:
#test
scrapehemi()

{'Cerberus Hemisphere Enhanced': '/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
 'Schiaparelli Hemisphere Enhanced': '/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
 'Syrtis Major Hemisphere Enhanced': '/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
 'Valles Marineris Hemisphere Enhanced': '/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}

In [35]:
#compressing all the scraping functions into one big function, and compiling the value as a dictionary
def scrape():
    scrapeDict = {}
    newslist = scrapenews()
    hemilist = scrapehemi()
    scrapeDict["mars_news_title"] = newslist[0]
    scrapeDict["mars_news_summary"] = newslist[1]
    scrapeDict["featured_image"] = scrapeimg()
    scrapeDict["mars_weather"] = scrapeweather()
    scrapeDict["hemisphere_images1"] = list(hemilist.values())[0]
    scrapeDict["hemisphere_images2"] = list(hemilist.values())[1]
    scrapeDict["hemisphere_images3"] = list(hemilist.values())[2]
    scrapeDict["hemisphere_images4"] = list(hemilist.values())[3]
    
    return scrapeDict



In [41]:
test = scrape()

AttributeError: 'NoneType' object has no attribute 'find'